In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile6 = pd.read_pickle("FFF_Train_Decile6.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile6.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile6, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 6')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 4s 1ms/step - loss: 0.3518 - acc: 0.8642 - val_loss: 0.3254 - val_acc: 0.8717
Epoch 39/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3501 - acc: 0.8619 - val_loss: 0.3282 - val_acc: 0.8665
Epoch 40/100
3055/3055 [==============================] - 5s 1ms/step - loss: 0.3480 - acc: 0.8619 - val_loss: 0.3245 - val_acc: 0.8691
Epoch 41/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3454 - acc: 0.8645 - val_loss: 0.3258 - val_acc: 0.8639
Epoch 42/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3498 - acc: 0.8632 - val_loss: 0.3269 - val_acc: 0.8704
Epoch 43/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3463 - acc: 0.8645 - val_loss: 0.3245 - val_acc: 0.8757
Epoch 44/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3454 - acc: 0.8632 - val_loss: 0.3220 - val_acc: 0.8783
Epoch 45/100
3055/3055 [==============================] - 4s 

Epoch 98/100
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3333 - acc: 0.8635 - val_loss: 0.3132 - val_acc: 0.8743
Epoch 99/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3322 - acc: 0.8687 - val_loss: 0.3133 - val_acc: 0.8730
Epoch 100/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3349 - acc: 0.8691 - val_loss: 0.3140 - val_acc: 0.8730
Test accuracy: 0.8730366482784611
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.5899 - acc: 0.6599 - val_loss: 0.4589 - val_acc: 0.8338
Epoch 2/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.4486 - acc: 0.8357 - val_loss: 0.3993 - val_acc: 0.8613
Epoch 3/150
3055/3055 [==============================] - 5s 1ms/step - loss: 0.4108 - acc: 0.8507 - val_loss: 0.3623 - val_acc: 0.8678
Epoch 4/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3840 - acc: 0.8550 - val_l

3055/3055 [==============================] - 5s 1ms/step - loss: 0.3126 - acc: 0.8730 - val_loss: 0.2976 - val_acc: 0.8796
Epoch 57/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3064 - acc: 0.8743 - val_loss: 0.2974 - val_acc: 0.8796
Epoch 58/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3098 - acc: 0.8730 - val_loss: 0.2974 - val_acc: 0.8796
Epoch 59/150
3055/3055 [==============================] - 5s 1ms/step - loss: 0.3061 - acc: 0.8750 - val_loss: 0.2973 - val_acc: 0.8796
Epoch 60/150
3055/3055 [==============================] - 5s 1ms/step - loss: 0.3138 - acc: 0.8736 - val_loss: 0.2972 - val_acc: 0.8796
Epoch 61/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3060 - acc: 0.8736 - val_loss: 0.2972 - val_acc: 0.8796
Epoch 62/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3081 - acc: 0.8753 - val_loss: 0.2971 - val_acc: 0.8796
Epoch 63/150
3055/3055 [==============================] - 5s 

3055/3055 [==============================] - 5s 2ms/step - loss: 0.3091 - acc: 0.8740 - val_loss: 0.2973 - val_acc: 0.8796
Epoch 117/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3032 - acc: 0.8723 - val_loss: 0.2973 - val_acc: 0.8796
Epoch 118/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3069 - acc: 0.8763 - val_loss: 0.2970 - val_acc: 0.8796
Epoch 119/150
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3133 - acc: 0.8792 - val_loss: 0.2971 - val_acc: 0.8796
Epoch 120/150
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3048 - acc: 0.8750 - val_loss: 0.2972 - val_acc: 0.8796
Epoch 121/150
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3088 - acc: 0.8736 - val_loss: 0.2974 - val_acc: 0.8796
Epoch 122/150
3055/3055 [==============================] - 6s 2ms/step - loss: 0.3084 - acc: 0.8763 - val_loss: 0.2969 - val_acc: 0.8796
Epoch 123/150
3055/3055 [==============================

...

3055/3055 [==============================] - 4s 1ms/step - loss: 0.3466 - acc: 0.8658 - val_loss: 0.3238 - val_acc: 0.8665

Epoch 00066: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 67/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3395 - acc: 0.8642 - val_loss: 0.3230 - val_acc: 0.8665
Epoch 68/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3440 - acc: 0.8625 - val_loss: 0.3227 - val_acc: 0.8665
Epoch 69/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3428 - acc: 0.8651 - val_loss: 0.3225 - val_acc: 0.8665
Epoch 70/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3405 - acc: 0.8661 - val_loss: 0.3229 - val_acc: 0.8652
Epoch 71/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3426 - acc: 0.8655 - val_loss: 0.3228 - val_acc: 0.8665
Epoch 72/100
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3426 - acc: 0.8655 - val_loss: 0.3223 - val_acc: 0.8

3055/3055 [==============================] - 4s 1ms/step - loss: 0.3164 - acc: 0.8691 - val_loss: 0.3078 - val_acc: 0.8678
Epoch 26/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3188 - acc: 0.8671 - val_loss: 0.3212 - val_acc: 0.8626
Epoch 27/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3144 - acc: 0.8789 - val_loss: 0.2952 - val_acc: 0.8796
Epoch 28/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3093 - acc: 0.8750 - val_loss: 0.2925 - val_acc: 0.8848
Epoch 29/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3154 - acc: 0.8717 - val_loss: 0.2968 - val_acc: 0.8848
Epoch 30/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3090 - acc: 0.8714 - val_loss: 0.2915 - val_acc: 0.8770
Epoch 31/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3087 - acc: 0.8736 - val_loss: 0.2888 - val_acc: 0.8848
Epoch 32/200
3055/3055 [==============================] - 4s 

3055/3055 [==============================] - 4s 1ms/step - loss: 0.2977 - acc: 0.8822 - val_loss: 0.2868 - val_acc: 0.8887
Epoch 84/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.2925 - acc: 0.8835 - val_loss: 0.2868 - val_acc: 0.8887
Epoch 85/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.2997 - acc: 0.8779 - val_loss: 0.2866 - val_acc: 0.8887
Epoch 86/200
3055/3055 [==============================] - 5s 2ms/step - loss: 0.2936 - acc: 0.8818 - val_loss: 0.2866 - val_acc: 0.8887
Epoch 87/200
3055/3055 [==============================] - 5s 1ms/step - loss: 0.2949 - acc: 0.8822 - val_loss: 0.2867 - val_acc: 0.8887
Epoch 88/200
3055/3055 [==============================] - 5s 1ms/step - loss: 0.2982 - acc: 0.8825 - val_loss: 0.2867 - val_acc: 0.8887
Epoch 89/200
3055/3055 [==============================] - 5s 1ms/step - loss: 0.2996 - acc: 0.8782 - val_loss: 0.2869 - val_acc: 0.8874
Epoch 90/200
3055/3055 [==============================] - 5s 

3055/3055 [==============================] - 4s 1ms/step - loss: 0.2966 - acc: 0.8841 - val_loss: 0.2866 - val_acc: 0.8887
Epoch 144/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.2953 - acc: 0.8831 - val_loss: 0.2866 - val_acc: 0.8887
Epoch 145/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.2978 - acc: 0.8822 - val_loss: 0.2865 - val_acc: 0.8887
Epoch 146/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.2988 - acc: 0.8805 - val_loss: 0.2865 - val_acc: 0.8887
Epoch 147/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.2912 - acc: 0.8802 - val_loss: 0.2864 - val_acc: 0.8887
Epoch 148/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.2975 - acc: 0.8786 - val_loss: 0.2864 - val_acc: 0.8887
Epoch 149/200
3055/3055 [==============================] - 4s 1ms/step - loss: 0.3004 - acc: 0.8838 - val_loss: 0.2864 - val_acc: 0.8887
Epoch 150/200
3055/3055 [==============================

3055/3055 [==============================] - 5s 1ms/step - loss: 0.5440 - acc: 0.6854 - val_loss: 0.4594 - val_acc: 0.7866
Epoch 3/50
3055/3055 [==============================] - 5s 1ms/step - loss: 0.4675 - acc: 0.7879 - val_loss: 0.4223 - val_acc: 0.8534
Epoch 4/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4370 - acc: 0.8396 - val_loss: 0.3964 - val_acc: 0.8678
Epoch 5/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.4045 - acc: 0.8507 - val_loss: 0.3584 - val_acc: 0.8586
Epoch 6/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3783 - acc: 0.8547 - val_loss: 0.3405 - val_acc: 0.8639
Epoch 7/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3731 - acc: 0.8566 - val_loss: 0.3342 - val_acc: 0.8691
Epoch 8/50
3055/3055 [==============================] - 5s 2ms/step - loss: 0.3619 - acc: 0.8606 - val_loss: 0.3393 - val_acc: 0.8717
Epoch 9/50
3055/3055 [==============================] - 5s 2ms/step - los

In [3]:
0.93484

0.93484

In [ ]:
#0.89